In [66]:
%pylab inline
from scipy.optimize import brentq
import pypropep as ppp
from pprint import pprint
ppp.init()

Populating the interactive namespace from numpy and matplotlib
Loaded 1921 thermo species
Loaded 1033 propellants


# Problem 1
Taking the balloon from your first project, let’s see how it would perform as a rocket with three different gas mixtures:
1. Nitrogen (close to air)
2. Oxygen + Hydrogen where the mole fraction of hydrogen is 0.95
3. Equimolar Deuterium + Tritium undergoing nuclear fusion

Our goal is to compute the specific impulse and ∆V for the balloon if released in the perfect vacuum of space (so exit pressure is zero and there is no drag).  Assume the balloon itself weighs 2g and is filled to 1.5 bar at 298K prior to being released.  Additional assumptions

- For all cases assume the nozzle naturally from a convergent/divergent nozzle with an area ratio Ae/At of 2.
- For the purposes of the nitrogen case, you may assume that the internal temperature and pressure of gas stays constant during the exhausting process. 
- For the purposes of both the oxygen + hydrogen and deuterium, tritium cases assume that once ignited the flame in the balloon propagates exactly such that the pressure in the balloon remains constant during exhaust process.  - Also assume that the incredible temperatures would not melt the balloon!
- For case (2) assume the combustion proceeds to stoichiometric completion and the only products are water and any excess hydrogen (H2) or oxygen (O2).  Also assume that the specific heat of water, hydrogen and oxygen are all constant of $C_p = 5 R$ J/K-mol. 
- For case (3) you may assume that the D-T fusion reaction liberates 17.6 MeV of energy per molecular fusion.  - You may also assume that this reaction proceeds to completion with the only product being Helium-4 with a constant specific heat of $C_p = 2.5 R$ J/K-mol.

After you have computed specific impulse for case (2), use one of the full chemical equilibrium codes to compute the same thing assuming frozen flow and an infinite contraction ratio chamber.  Compare the results to your hand calculations.

## Solution

First let's solve for $P_e/P_t$ given the area ratio provided.  Since we are exhausting to vacuum, we can safely assume $P_0 \approx 0$ and the nozzle is choked.  Given $f(M)$:

$$f(M) = \frac{A^*}{A} = \left[\frac{\gamma + 1}{2}\right]^{\frac{\gamma + 1}{2(\gamma-1)}}\frac{M}{\left[1 + \frac{\gamma-1}{2}M^2\right]^{\frac{\gamma + 1}{2(\gamma-1)}}}$$

We are given that $\frac{A_e}{A^*} = 2$ so solve $f(M)$ for M. at the various $\gamma$'s.

In [67]:
Ae_As = 2.
def f(M, k, f_targ):
    return f_targ - (((k + 1.) / 2.)**((k + 1.) / (2. * (k - 1.))) * M /
            (1. + (k - 1.)/2. * M**2)**((k + 1.) / (2. * (k - 1.))))

gamma = np.array([1.4, 1.4, 1.66])
M = np.zeros_like(gamma)

for i, g in enumerate(gamma):
    M[i] = brentq(f, 1.0, 10., args=(g, 1./Ae_As))

Pe_Pt = (1. + (gamma-1)/2 * M**2)**(-gamma / (gamma - 1))
print Pe_Pt

[0.09393265 0.09393265 0.06904846]


Now let's look at the $C$ associated with each system by computing a stagnation temperature.

### Case 1 (Nitrogen)
We know nitrogen is 298K so we're done there.

$c^* = \frac{P_t A^*}{\dot{m}} = \sqrt{\frac{R_u T_t}{\gamma M_w}}\left[\frac{\gamma + 1}{2}\right]^{\frac{\gamma + 1}{2(\gamma - 1)}}$

$C_f = \frac{T}{P_t A^*} = \frac{\left(\frac{\gamma + 1}{2}\right)^{\frac{\gamma + 1}{-2(\gamma-1)}}}{M_e \sqrt{1+\frac{\gamma - 1}{2}M_e^2}}\left[\gamma M_e^2 + 1 - \frac{P_0}{P_e}\right]$

$C = C_f  c^*$

In [72]:
Ru = 8.314  # J / mol - K
def cstar(Tt, Mw, gamma):
    return (np.sqrt(Ru/Mw * Tt / gamma) * 
            ((gamma + 1)/2)**((gamma + 1)/2/(gamma-1)))

def Cf(Me, P0_Pe, gamma):
    return (((gamma + 1) / 2)**((gamma + 1)/-2/(gamma -1)) / Me /
            np.sqrt(1 + (gamma -1)/2*Me**2) * (gamma * Me**2 + 1 - P0_Pe))

def C(Tt, Mw, Pe_Pt, Ae_As, gamma):
    Me = brentq(f, 1.0, 10., args=(gamma, 1./Ae_As))
    return cstar(Tt, Mw, gamma) * Cf(Me, 0.0, gamma)

C1 = C(298., 0.028, Pe_Pt[0], Ae_As, gamma[0])
print "Case 1 C: %.1f m/s" % C1

Case 1 C: 633.2 m/s


### Case 2 (O2 + H2)
For case (2) we know $X_{H_2} = 0.95 = \frac{n_{H_2}}{n_{H_2} + n_{O_2}}$ and therefore $n_{H_2} = 19 n_{O_2}$
Now we can write the reaction equation:

$$19 H_{2} (g) + O_2 (g) \rightarrow 2 H_{2}O (g) + 17 H_{2} + \Delta H_{rxn}$$

$$\Delta H_{rxn} = \left[2 \Delta_f H^\circ(T_r)_{H_2O} + 17 \Delta_f H^\circ(T_r)_{H_2}\right] -
\left[19 \Delta_f H^\circ(T_r)_{H_2} + \Delta_f H^\circ(T_r)_{O_2}\right]$$

- $\Delta_f H^\circ(T_r)_{H_2} = 0$ kJ/mol (reference)
- $\Delta_f H^\circ(T_r)_{O_2} = 0$ kJ/mol (reference)
- $\Delta_f H^\circ(T_r)_{H_2O} = -241.826$ kJ/mol

$\Rightarrow \Delta H_{rxn} = -483.7$ kJ

Given that we assume the specific heat of all the consitituents are 5 $R_u$ J / mol - K, we can show that

$$T_t - T_r = \frac{-\Delta H_{rxn}}{n_{prod} C_p} = \frac{483.7\times 10^3 \text{J}}{19 \times 5 \times R_u} = 612 K$$

We now have what we need to compute $C$

In [87]:
Cp_R = 5.
Tr = 298.15                #K
delFH_H2O = -241.826e3
Tt = -2 * delFH_H2O / 19. / Cp_R / Ru + Tr
Mw_final = ( (2. * .018 + 17. * .002) / 
            (2. + 17.) )

C2 = C(Tt, Mw_final, Pe_Pt[1], Ae_As, gamma[1])
print "Case 2 C: %.1f m/s" % C2

p = ppp.FrozenPerformance()
o2 = ppp.PROPELLANTS['OXYGEN (GAS)']
h2 = ppp.PROPELLANTS['HYDROGEN (GASEOUS)']
p.add_propellants([(h2, 19.0), (o2, 1.0)])
p.set_state(P=1.5, Ae_At=2)
print p.performance.Ivac

Case 2 C: 3051.3 m/s
3430.68549773


## Using the degree of freedom method

If we recognize that hydrogen gas is diatomic and is hte dominent component, we can use the degree of freedom method to estimate $C_p$.  $\beta = 5$ (3 translation + 2 rotation) so $C_p = \frac{5 + 2}{2}R_u$.

In [90]:
Cp_R = 7./2
Tr = 298.15                #K
delFH_H2O = -241.826e3
Tt = -2 * delFH_H2O / 19. / Cp_R / Ru + Tr
Mw_final = ( (2. * .018 + 17. * .002) / 
            (2. + 17.) )

C2 = C(Tt, Mw_final, Pe_Pt[1], Ae_As, gamma[1])
print "Case 2 (estimated C_p) C: %.1f m/s" % C2

Case 2 (estimated C_p) C: 3463.3 m/s


**This is really close!!**

### Case 3 (Tritium, Deuterium)

For this case we have to compute the effective heat of reaction for the tritium + deuterium fusion reaction:

$$^3H_2 (g) + ^2H_2(g) \rightarrow ^4He (3.5 \text{MeV}) + ^1n (14.1 \text{MeV})$$

For this we are assuming that the balloon can magically thermalize the neutrons and alpha particles (helium) such that all of their resulting kinetic energy is available to heat the helium (this of course is not true).  This provides 17.6 MeV of energy per reaction or 1,717 GJ / (mol Helium) produced.  And we can write

$$T_t - T_r = \frac{1.717\times 10^{12} \text{J}}{2.5 \times R_u} = 8.3 \times 10^{10} K$$

And finally

In [78]:
C3 = C(8.3e10, .004, Pe_Pt[2], Ae_As, gamma[2])
print "Case 3 C: %.1f m/s" % C3

Case 3 C: 26333202.6 m/s


## Delta-V's

To compute delta-V we must know initial mass and final masses.  We are given balloon mass so we need to just compute the gas masses.  We will assume the radius of the balloon fully inflated is 10cm.  Now we can compute the gas masses.

In [88]:
r = 10e-2                 # cm
V = 4. * pi / 3. * r**3   # m^3
Pt = 1.5e5                # Pa
T0 = 298.                 # K
m_balloon = .002          # kg

# Case 1
MW_n2 = .028              # kg / mol
rho_n2 = Pt / (Ru / MW_n2) / T0
m_n2 = V * rho_n2
Mi_Mf_1 = (m_n2 + m_balloon) / (m_balloon)
deltaV_1 = C1 * np.log(Mi_Mf_1)
print "Case 1 DeltaV: %.3f km/s" % (deltaV_1 / 1e3)

# Case 2
MW_react = (19. * .002 + 1. * .032) / (19. + 1.)              # kg / mol
rho_react = Pt / (Ru / MW_react) / T0
m_react = V * rho_react
Mi_Mf_2 = (m_react + m_balloon) / (m_balloon)
deltaV_2 = C2 * np.log(Mi_Mf_2)
print "Case 2 DeltaV: %.3f km/s" % (deltaV_2 / 1e3)

# Case 3
MW_react_DT = (1. * .006 + 1. * .004) / (2.)              # kg / mol
rho_react_DT = Pt / (Ru / MW_react_DT) / T0
m_react_DT = V * rho_react_DT
Mi_Mf_3 = (m_react_DT + m_balloon) / (m_balloon)
deltaV_3 = C3 * np.log(Mi_Mf_3)
print "Case 3 DeltaV: %.3f km/s" % (deltaV_3 / 1e3)
print "Case 3 v/c: %.3f" % (deltaV_3 / 300e6)

Case 1 DeltaV: 0.959 km/s
Case 2 DeltaV: 1.121 km/s
Case 3 DeltaV: 12930.521 km/s
Case 3 v/c: 0.043


## 2 NO2 <-> N2O4 Equilibrium

From JANAF tables...


In [49]:
P_Pstd = 1.
T = [250, 298.15, 350, 600]
log_K_f0_NO2 = [-10.103, -8.980, -8.125, -6.114]
log_K_f0_N2O4 = [-17.449, -17.132, -16.902, -16.359]

for i in range(len(T)):
    Kp = 10**(log_K_f0_N2O4[i] - 2 * log_K_f0_NO2[i] - np.log10(P_Pstd))
    X_NO2 = (np.sqrt(4 * Kp + 1) - 1) / 2 / Kp
    X_N2O4 = 1 - X_NO2
    print "X_NO2: %.3f, X_N2O4: %.3f" % (X_NO2, X_N2O4)

X_NO2: 0.041, X_N2O4: 0.959
X_NO2: 0.318, X_N2O4: 0.682
X_NO2: 0.842, X_N2O4: 0.158
X_NO2: 1.000, X_N2O4: 0.000
